In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords})


In [3]:
# Assuming property_gdf and victoria_gdf are already defined as per your provided code...
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/2101387466.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [4]:
station = gpd.read_file('../data/external_SA2/Station.geojson')

In [5]:
station_SA2 = gpd.sjoin(station, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
property_agg = pd.merge(property_SA2, station_SA2, on='SA2_CODE21', how='left')

In [7]:
merged = property_agg[['SA2_CODE21','geometry_x', 'geometry_y']]

In [8]:
merged.rename(columns={'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/2385872553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)


In [9]:
park = gpd.read_file('../data/external_SA2/Park.geojson')

In [10]:
park_SA2 = gpd.sjoin(park, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [11]:
property_agg1 = pd.merge(merged, park_SA2, on='SA2_CODE21', how='left')

In [12]:
merged = property_agg1[['SA2_CODE21','property_geometry', 'station_geometry', 'geometry']]
merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/2976054265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)


In [13]:
shop = gpd.read_file('../data/external_SA2/Mall, Shopping Centre & Department Store.geojson')

In [14]:
shop_SA2 = gpd.sjoin(shop, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [15]:
property_agg2 = pd.merge(merged, shop_SA2, on='SA2_CODE21', how='left')

In [16]:
merged = property_agg2[['SA2_CODE21','property_geometry', 'station_geometry', 'park_geometry', 'geometry']]
merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/256104151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)


In [17]:
hospital = gpd.read_file('../data/external_SA2/Hospital.geojson')

In [18]:
hospital_SA2 = gpd.sjoin(hospital, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [19]:
property_agg3 = pd.merge(merged, hospital_SA2, on='SA2_CODE21', how='left')

In [20]:
merged = property_agg3[['SA2_CODE21','property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry', 'geometry']]
merged.rename(columns={'geometry': 'hospital_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/3350325718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'hospital_geometry'}, inplace=True)


In [21]:
school_df = pd.read_csv("../data/external_SA2/complete_school_location.csv", encoding='latin-1')
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [22]:
# Your property data
school_metadata = school_df

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
#school_metadata['coordinates'] = school_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [23]:
school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/1493813974.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")


In [24]:
property_agg4 = pd.merge(merged, school_SA2, on='SA2_CODE21', how='left')

In [25]:
merged = property_agg4[['SA2_CODE21','property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry','hospital_geometry', 'geometry']]
merged.rename(columns={'geometry': 'school_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/2215113417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'school_geometry'}, inplace=True)


In [26]:
supermarket = gpd.read_file('../data/external_SA2/Supermarket.geojson')
supermarket_SA2 = gpd.sjoin(supermarket, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [27]:
property_agg5 = pd.merge(merged, supermarket_SA2, on='SA2_CODE21', how='left')

In [28]:
merged = property_agg5[['SA2_CODE21','property_geometry', 'station_geometry', 'park_geometry', 'shop_geometry','hospital_geometry','school_geometry', 'geometry']]
merged.rename(columns={'geometry': 'supermarket_geometry'}, inplace=True)

/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_58065/3181743012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'supermarket_geometry'}, inplace=True)


In [30]:
merged

,SA2_CODE21,property_geometry,station_geometry,park_geometry,shop_geometry,hospital_geometry,school_geometry,supermarket_geometry
0,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279),None,POINT (144.97215 -37.81031),POINT (144.96497 -37.81756)
1,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279),None,POINT (144.97215 -37.81031),POINT (144.97115 -37.80989)
2,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279),None,POINT (144.97215 -37.81031),POINT (144.96749 -37.81211)
3,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279),None,POINT (144.97215 -37.81031),POINT (144.96510 -37.81795)
4,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279),None,POINT (144.97215 -37.81031),POINT (144.96453 -37.81667)
...,...,...,...,...,...,...,...,...
258441,212031555,POINT (145.35015 -38.08134),None,None,None,None,POINT (145.37283 -38.08273),None
258442,212031555,POINT (145.35015 -38.08134),None,None,None,None,POINT (145.36380 -38.08468),None
258443,212031555,POINT (145.34202 -38.08875),None,None,None,None,POINT (145.36439 -38.08443),None
258444,212031555,POINT (145.34202 -38.08875),None,None,None,None,POINT (145.37283 -38.08273),None


In [29]:
merged.to_csv("../data/raw/aggregated_locations.csv")